# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [14]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
units = "imperial"

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

632

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# city_df = pd.DataFrame(cities).rename(columns={0:"city_name"})
# city_df['city_longitude'] = ""
# city_df['city_latitude'] = ""
# city_df['max_temp'] = ""
# city_df['humidity'] = ""
# city_df['cloud_covg'] = ""
# city_df['wind_speed'] = ""
# city_df['city_country'] = ""
# city_df['city_datetime'] = ""
# city_df.head()


,city_name,city_longitude,city_latitude,max_temp,humidity,cloud_covg,wind_speed,city_country,city_datetime
0,tuatapere,,,,,,,,
1,carnarvon,,,,,,,,
2,goderich,,,,,,,,
3,hasaki,,,,,,,,
4,avarua,,,,,,,,


In [15]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"

In [5]:
# city_number = 0

# for index, row in city_df.iterrows():
#     c_name = row['city_name']
#     q_url = f"{base_url}q={c_name}&units={units}&appid={weather_api_key}"

#     weather_resp = requests.get(q_url).json()

#     try:
#         city_number += 1
#         city_df.loc[:, 'city_longitude'] = weather_resp['coord']['lon']
#         print(f"Processing Record {city_number} of {len(cities)}: {city}")


#     except (KeyError, IndexError):
#         print("City not found. Skipping...")

In [16]:
name = []
lng = []
lat = []
maxtemp = []
humid = []
cloud = []
windspd = []
country = []
datetime = []

city_number = 0

for city in cities[:10]:
    q_url = f"{base_url}q={city}&units={units}&appid={weather_api_key}"
    weather_resp = requests.get(q_url).json()

    try:
        city_number += 1
        
        name.append(weather_resp['name'])
        lng.append(weather_resp['coord']['lon'])
        lat.append(weather_resp['coord']['lat'])
        maxtemp.append(weather_resp['main']['temp_max'])
        humid.append(weather_resp['main']['humidity'])
        cloud.append(weather_resp['clouds']['all'])
        windspd.append(weather_resp['wind']['speed'])
        country.append(weather_resp['sys']['country'])
        datetime.append(weather_resp['dt'])
        

        print(f"Processing Record {city_number} of {len(cities)}: {city}")


    except (KeyError, IndexError):
        print("City not found. Skipping...")
        pass

Processing Record 1 of 632: lavrentiya
Processing Record 2 of 632: te anau
Processing Record 3 of 632: mataura
Processing Record 4 of 632: tateyama
Processing Record 5 of 632: port lincoln
Processing Record 6 of 632: barrow
Processing Record 7 of 632: kwinana
Processing Record 8 of 632: punta arenas
Processing Record 9 of 632: castro
Processing Record 10 of 632: severnyy


In [ ]:
# city_dict = {}
# city_number = 0

# for city in cities:
#     q_url = f"{base_url}q={city}&units={units}&appid={weather_api_key}"
#     weather_resp = requests.get(q_url).json()

#     try:
#         city_number += 1
        
#         city_df.loc[:, 'city_longitude'] = weather_resp['coord']['lon']
#         print(f"Processing Record {city_number} of {len(cities)}: {city}")


#     except (KeyError, IndexError):
#         print("City not found. Skipping...")

In [17]:
city_summary = pd.DataFrame({"City Name": name,
                            "Longitude": lng,
                            "Latitude": lat,
                            "Max Temperature": maxtemp,
                            "Humidtiy": humid,
                            "Cloud Coverage": cloud,
                            "Wind Speed": windspd,
                            "City Country": country,
                            "Date Time": datetime})
city_summary

,City Name,Longitude,Latitude,Max Temperature,Humidtiy,Cloud Coverage,Wind Speed,City Country,Date Time
0,Lavrentiya,-171.00,65.58,46.62,83,2,1.28,RU,1594314881
1,Te Anau,167.72,-45.42,36.34,87,43,1.03,NZ,1594314881
2,Mataura,168.86,-46.19,41.74,92,94,5.35,NZ,1594314881
3,Tateyama,139.87,34.98,73.99,100,75,10.29,JP,1594314881
4,Port Lincoln,135.87,-34.73,54.63,57,100,13.82,AU,1594314882
5,Barrow,-156.79,71.29,37.40,86,1,10.29,US,1594314882
6,Kwinana,115.77,-32.25,59.00,100,4,9.10,AU,1594314882
7,Punta Arenas,-70.92,-53.15,35.60,74,75,23.04,CL,1594314674
8,Castro,-50.01,-24.79,59.56,49,92,5.55,BR,1594314549
9,Severnyy,36.55,50.68,69.80,46,5,13.42,RU,1594314883


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression